# Single Objective Optimization with Credit Risk Analysis 

In [ ]:
%%capture
#Install dependencies

#For Synthetic Data Generation
!pip install sdv
from sdv.tabular import GaussianCopula

from io import StringIO
import os
import time
import sys
import IPython
from time import gmtime, strftime

import boto3
import numpy as np
import pandas as pd
import urllib

import sagemaker
from sagemaker.s3 import S3Uploader
from sagemaker.processing import ProcessingInput, ProcessingOutput
from sagemaker.sklearn.processing import SKLearnProcessor
from sagemaker.inputs import TrainingInput
from sagemaker.xgboost import XGBoost
from sagemaker.s3 import S3Downloader
from sagemaker.s3 import S3Uploader
from sagemaker import Session
from sagemaker import get_execution_role
from sagemaker.xgboost import XGBoostModel
from sagemaker.sklearn import SKLearnModel
from sagemaker.pipeline import PipelineModel
from sagemaker.inputs import TrainingInput


session = Session()
bucket = session.default_bucket()
prefix = "sagemaker/sagemaker-amt-credit-risk-model"
region = session.boto_region_name

# Define IAM role
role = get_execution_role()

In [ ]:
#Retrieve Biased dataset from Notebook 1 
%store -r BiasedData1

In [ ]:
training_data = BiasedData1 
training_data.info()
BiasedData1.head()

### Create the raw training and test CSV files

In [ ]:
# prepare raw test data
test_data = training_data.sample(frac=0.1)
test_data = test_data.drop(["credit_risk"], axis=1)
test_filename = "test.csv"
test_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
]
test_data.to_csv(test_filename, index=False, header=True, columns=test_columns, sep=",")

# prepare raw training data
credit_columns = [
    "status",
    "duration",
    "credit_history",
    "purpose",
    "amount",
    "savings",
    "employment_duration",
    "installment_rate",
    "personal_status_sex",
    "other_debtors",
    "present_residence",
    "property",
    "age",
    "other_installment_plans",
    "housing",
    "number_credits",
    "job",
    "people_liable",
    "telephone",
    "foreign_worker",
    "credit_risk",
]
train_filename = "train.csv"
training_data.to_csv(train_filename, index=False, header=True, columns=credit_columns, sep=",")

### Encode and Upload Data
Here we encode the training and test data. Encoding input data is not necessary for SageMaker Clarify, but is necessary for XGBoost models.

In [ ]:
test_raw = S3Uploader.upload(test_filename, "s3://{}/{}/data/test".format(bucket, prefix))
(test_raw)

In [ ]:
train_raw = S3Uploader.upload(train_filename, "s3://{}/{}/data/train".format(bucket, prefix))
print(train_raw)

### Preprocessing and feature engineering with SageMaker Processing job

We will use SageMaker Processing jobs to perform the preprocessing on the raw data. SageMaker Processing provides prebuilt container for SKlearn which we will use here. We will output a sklearn model that can be used for preprocessing inference requests. 

In [ ]:
sklearn_processor = SKLearnProcessor(
    role=role,
    base_job_name="sagemaker-amt-credit-risk-processing-job",
    instance_type="ml.m5.large",
    instance_count=1,
    framework_version="0.20.0",
)

You can have a look at the preprocessing script prepared to run in the processing job

In [ ]:
# !pygmentize processing/preprocessor.py

#### NOTE: THIS CELL WILL RUN FOR APPROX. 5-8 MINUTES! PLEASE BE PATIENT. 
For further documentation on SageMaker Processing, you can refer the documentation [here](https://docs.aws.amazon.com/sagemaker/latest/dg/processing-job.html)

In [ ]:
raw_data_path = "s3://{0}/{1}/data/train/".format(bucket, prefix)
train_data_path = "s3://{0}/{1}/data/preprocessed/train/".format(bucket, prefix)
val_data_path = "s3://{0}/{1}/data/preprocessed/val/".format(bucket, prefix)
model_path = "s3://{0}/{1}/sklearn/".format(bucket, prefix)


sklearn_processor.run(
    code="processing/preprocessor.py",
    inputs=[
        ProcessingInput(
            input_name="raw_data", source=raw_data_path, destination="/opt/ml/processing/input"
        )
    ],
    outputs=[
        ProcessingOutput(
            output_name="train_data", source="/opt/ml/processing/train", destination=train_data_path
        ),
        ProcessingOutput(
            output_name="val_data", source="/opt/ml/processing/val", destination=val_data_path
        ),
        ProcessingOutput(
            output_name="model", source="/opt/ml/processing/model", destination=model_path
        ),
    ],
    arguments=["--train-test-split-ratio", "0.2"],
    logs=False,
)

## Train XGBoost Model
In this step, we will train an XGBoost model on the preprocessed data. We will use our own training script with the built-in XGBoost container provided by SageMaker.

Alternatively, for your own use case, you can also bring your own model (trained elsewhere) to SageMaker for processing with SageMaker Clarify


In [ ]:
# !pygmentize training/train_xg_amt.py

### Set up XGBoost Estimator

Next, let us set up:    
 1. Pre-defined values for Hyperparameters for XGBoost algorithm
 1. XGBoost Estimator for SageMaker

    

In [ ]:
hyperparameters = {
    "max_depth": "5",
    "eta": "0.1",
    "gamma": "4",
    "min_child_weight": "6",
    "silent": "1",
    "objective": "binary:logistic",
    "num_round": "100",
    "subsample": "0.8",
    "eval_metric": "auc",
    "early_stopping_rounds": "20",
    "output_data_dir": "/opt/ml/output/data/",
}

entry_point = "train_xg_amt_single.py"
source_dir = "training/"
output_path = "s3://{0}/{1}/{2}".format(bucket, prefix, "xgb_model")
code_location = "s3://{0}/{1}/code".format(bucket, prefix)

estimator = XGBoost(
    entry_point=entry_point,
    source_dir=source_dir,
    output_path=output_path,
    code_location=code_location,
    hyperparameters=hyperparameters,
    instance_type="ml.c5.xlarge",
    instance_count=1,
    framework_version="0.90-2",
    py_version="py3",
    role=role,
)

### SageMaker AMT (HPO)

Now it's time to run the HPO job to train and find the best model 

#### NOTE: THIS CELL WILL RUN FOR APPROX. 5-8 MINUTES! PLEASE BE PATIENT.

In [ ]:
# Automatic Model Tuning (HPO)

# output_data_dir = 's3://sagemaker-us-east-2-921553072635/sagemaker/sagemaker-amt-credit-risk-model/data/output/'

from sagemaker.tuner import IntegerParameter, CategoricalParameter, ContinuousParameter, HyperparameterTuner

hyperparameter_ranges = {'eta': ContinuousParameter(0, 1),
                        'min_child_weight': IntegerParameter(1, 10),
                        'gamma': IntegerParameter(1, 5),
                        'max_depth': IntegerParameter(1, 10)}

objective_metric_name = 'validation:auc'

tuner = HyperparameterTuner(estimator,
                            objective_metric_name,
                            hyperparameter_ranges,
                            max_jobs=500, # 500
                            max_parallel_jobs=10, #16
                           )

tuning_job_name = "xgb-tuner-{}".format(strftime("%d-%H-%M-%S", gmtime()))

# inputs = {'train': train_data_path, 'validation': val_data_path, 'output_data_dir': output_data_dir}

inputs = {'train': train_data_path, 'validation': val_data_path}

tuner.fit(inputs, job_name=tuning_job_name)

tuner.wait()

tuner_metrics = sagemaker.HyperparameterTuningJobAnalytics(tuning_job_name)

### The best Area Under Curve (AUC) score, alongside the Disparate Impact was found by going into the SageMaker logs where they are saved. You can find the logs by following the image guide below

In the SageMaker console page navigate the left menu and find Training -> Hyperparameter Tuning Jobs. That will bring up all the recent Tuuning jobs and you can click the latest one. 
![Step1](images/Step1A.jpg)

Scrolling down the tuning job page will bring you to “Best training job summary”. Here you can see the name of the best training job and the optimized value of the metric specified. In this case you can see that the AUC value is optimized at 0.91. Click on the best training job name.
![Step2](images/Step2A.jpg)

Scrolling down the page of the best training job, you will find a section under “Monitor” called “View logs”. Click on this to take you to CloudWatch logs.
![Step3](images/Step3A.jpg)

Scrolling down this current page you will encounter a section called “Log streams”. Click the name of your training job under this section.
![Step4](images/Step4A.jpg)

On this current page we can scroll all the way down and view all of the values the AI model has tried, the final values are at the bottom and are the optimized ones. 
![Step5](images/Step5A.jpg)

In [ ]:
#Best AUC Score for single Metric Optimization 
auc_score = 0.91
#Acompanying Disparate Impact
di = 0.40

### As can be seen we have a very high AUC Score which is due to the optimization of the tuning job. The tuning was not optimized for DI therefor the value is not the best (lowest) as can be. Please refer back to these values as you will require them to complete the lab in the next notebook.  

### You may now move onto "4-Multi-AMT.ipynb" to explore Multi Objective Optimization tuning jobs to optimize both AUC & DI